In [39]:
# Importing necessary libraries 
import os
import json
import requests
from astropy.table import Table
from urllib.parse import urlencode

### General Structure of Reading JSON files from SLED and FITS Imaging Downloads from Multiple Databases.

In [40]:
def load_sled_table(json_file):
    """
    Load JSON file of lensed objects into an Astropy Table.
    Each row corresponds to a unique 'name' entry in the JSON.
    """
    with open(json_file, "r") as f:
        data = json.load(f)

    rows = []
    for entry in data:
        rows.append({
            "name": entry.get("name", "Unknown"),
            "ra": float(entry.get("ra", None)),
            "dec": float(entry.get("dec", None)),
            "score": entry.get("score", None),
            "flag": entry.get("flag", "Unknown"),
            "n_img": entry.get("n_img", None),
            "image_sep": entry.get("image_sep", None),
            "lens_type": entry.get("lens_type", "Unknown"),
            "source_type": entry.get("source_type", "Unknown"),
            "imaging": entry.get("imaging", [])
        })

    table = Table(
        rows=rows,
        names=[
            "name", "ra", "dec", "score",
            "flag", "n_img", "image_sep", "lens_type", "source_type", "imaging"
        ]
    )
    return table

In [41]:
def download_fits(url, outpath):
    """Download a FITS file and save it to outpath."""
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        with open(outpath, "wb") as f:
            f.write(response.content)
        print(f"Saved {outpath}")
    except requests.HTTPError as e:
        print(f"Failed to fetch {url}: {e}")

In [42]:
def query_panstarrs(ra, dec, imaging, outdir, name):
    """
    Download Pan-STARRS FITS cutouts for each band in imaging using ps1filenames.py service.
    Saved as: ./outdir/<name>/<instrument>/<name>_<band>.fits
    """
    base_fitscut = "https://ps1images.stsci.edu/cgi-bin/fitscut.cgi?"
    valid_bands = {"g","r","i","z","y"}

    for img in imaging:
        instrument = img.get("instrument")
        if instrument != "Pan-STARRS1":
            continue

        band = img.get("band")
        if not band:
            print(f"Skipping {name}: missing band")
            continue
        band = band.strip().lower()
        if band not in valid_bands:
            print(f"Skipping {name}: invalid band {band}")
            continue

        # Query list of filenames for this position & filter
        try:
            url_table = f"https://ps1images.stsci.edu/cgi-bin/ps1filenames.py?ra={ra}&dec={dec}&filters={band}"
            table = Table.read(url_table, format='ascii')
            if len(table) == 0:
                print(f"No PS1 images found for {name} band {band}")
                continue
            filename = table['filename'][0]  # take first available image
        except Exception as e:
            print(f"Failed to query filenames for {name} band {band}: {e}")
            continue

        # Always use size = 120 pixels
        size_pixels = 120

        params = {
            "ra": ra,
            "dec": dec,
            "size": size_pixels,
            "format": "fits",
            "red": filename  # single-band FITS uses red param
        }

        url = base_fitscut + urlencode(params)

        inst_dir = os.path.join(outdir, name, instrument)
        os.makedirs(inst_dir, exist_ok=True)
        outpath = os.path.join(inst_dir, f"{name}_{band}.fits")

        try:
            download_fits(url, outpath)
        except Exception as e:
            print(f"Failed to fetch {url}: {e}")

### Accessing JSON lensed galaxies file

### Accessing JSON lensed quasars file 

### Accessing JSON lensed supernovae file 


In [43]:
if __name__ == "__main__":
    json_path = "./lensed_supernovae/supernovae.json"
    outdir = "./lensed_supernovae/cutouts"

    # Load JSON catalog into an Astropy Table
    table = load_sled_table(json_path)
    print(table)

    # Loop through catalog entries and fetch Pan-STARRS cutouts
    for row in table:
        name = row["name"]
        ra = row["ra"]
        dec = row["dec"]
        imaging_list = row["imaging"]

        if not imaging_list:
            print(f"No imaging data for {name}, skipping.")
            continue

        query_panstarrs(
            ra=ra,
            dec=dec,
            imaging=imaging_list,
            outdir=outdir,
            name=name
        )

     name      ...
-------------- ...
    J0138-2155 ...
 ABELL370_GLSN ...
MACSJ1149_GLSN ...
    SN-2022qmx ...
     iPTF16geu ...
Saved ./lensed_supernovae/cutouts\J0138-2155\Pan-STARRS1\J0138-2155_y.fits
Saved ./lensed_supernovae/cutouts\J0138-2155\Pan-STARRS1\J0138-2155_r.fits
Saved ./lensed_supernovae/cutouts\J0138-2155\Pan-STARRS1\J0138-2155_z.fits
Saved ./lensed_supernovae/cutouts\J0138-2155\Pan-STARRS1\J0138-2155_g.fits
Saved ./lensed_supernovae/cutouts\J0138-2155\Pan-STARRS1\J0138-2155_i.fits
Saved ./lensed_supernovae/cutouts\ABELL370_GLSN\Pan-STARRS1\ABELL370_GLSN_z.fits
Saved ./lensed_supernovae/cutouts\ABELL370_GLSN\Pan-STARRS1\ABELL370_GLSN_r.fits
Saved ./lensed_supernovae/cutouts\ABELL370_GLSN\Pan-STARRS1\ABELL370_GLSN_g.fits
Saved ./lensed_supernovae/cutouts\ABELL370_GLSN\Pan-STARRS1\ABELL370_GLSN_i.fits
Saved ./lensed_supernovae/cutouts\ABELL370_GLSN\Pan-STARRS1\ABELL370_GLSN_y.fits
Saved ./lensed_supernovae/cutouts\MACSJ1149_GLSN\Pan-STARRS1\MACSJ1149_GLSN_y.fits
Save